<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [1]</a>'.</span>

# Initiate spark

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [1]:
import os
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

conf = (
    SparkConf()
    .setAppName("knmi daily weather to bronze")
    .set("spark.hadoop.fs.s3a.endpoint", "http://192.168.86.192:9000")
    .set("spark.hadoop.fs.s3a.access.key", os.getenv('MINIO_ROOT_USER'))
    .set("spark.hadoop.fs.s3a.secret.key", os.getenv('MINIO_ROOT_PASSWORD'))
    .set("spark.hadoop.fs.s3a.path.style.access", True)
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .set("spark.driver.memory", "8g")
    .set("spark.executor.memory", "8g")
    .set("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") 
    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
    .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 
)
sc = SparkContext(conf=conf).getOrCreate()
#sqlContext = SQLContext(sc)
spark = SparkSession(sc).builder.getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
schema = (StructType([
    StructField("STN", StringType(), True),
    StructField("YYYYMMDD", StringType(), True),
    StructField("DDVEC", StringType(), True),
    StructField("FHVEC", StringType(), True),
    StructField("FG", StringType(), True),
    StructField("FHX", StringType(), True),
    StructField("FHXH", StringType(), True),
    StructField("FHN", StringType(), True),
    StructField("FHNH", StringType(), True),
    StructField("FXX", StringType(), True),
    StructField("FXXH", StringType(), True),
    StructField("TG", StringType(), True),
    StructField("TN", StringType(), True),
    StructField("TNH", StringType(), True),
    StructField("TX", StringType(), True),
    StructField("TXH", StringType(), True),
    StructField("T10N", StringType(), True),
    StructField("T10NH", StringType(), True),
    StructField("SQ", StringType(), True),
    StructField("SP", StringType(), True),
    StructField("Q", StringType(), True),
    StructField("DR", StringType(), True),
    StructField("RH", StringType(), True),
    StructField("RHX", StringType(), True),
    StructField("RHXH", StringType(), True),
    StructField("PG", StringType(), True),
    StructField("PX", StringType(), True),
    StructField("PXH", StringType(), True),
    StructField("PN", StringType(), True),
    StructField("PNH", StringType(), True),
    StructField("VVN", StringType(), True),
    StructField("VVNH", StringType(), True),
    StructField("VVX", StringType(), True),
    StructField("VVXH", StringType(), True),
    StructField("NG", StringType(), True),
    StructField("UG", StringType(), True),
    StructField("UX", StringType(), True),
    StructField("UXH", StringType(), True),
    StructField("UN", StringType(), True),
    StructField("UNH", StringType(), True),
    StructField("EV24", StringType(), True)
])
)

In [ ]:
df = spark.read.csv('s3a://landing-knmi/daggegevens/*', header=False, comment='#', schema=schema)

In [ ]:
from pyspark.sql.functions import current_timestamp
df = df.withColumn('load_datetime', current_timestamp())
df.show(3)

In [ ]:
df.dropDuplicates().createOrReplaceTempView('daggegevens')

In [ ]:
#df.dropDuplicates().coalesce(4).write.format("delta").mode("overwrite").option("mergeSchema", "true").save('s3a://bronze-knmi/daggegevens')
df_deduplicated = df.dropDuplicates()

In [ ]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, "s3a://bronze-knmi/daggegevens")

In [ ]:
deltaTable.alias('knmi_bronze_daggegevens') \
  .merge(
    df_deduplicated.alias('new_data'),
    'knmi_bronze_daggegevens.STN = new_data.STN AND knmi_bronze_daggegevens.YYYYMMDD = new_data.YYYYMMDD'
  ) \
  .whenNotMatchedInsertAll() \
  .execute()

In [ ]:
spark.sql('''

select STN, YYYYMMDD, load_datetime from daggegevens where YYYYMMDD = '20220101' order by STN

''').show(100)